# Preparing Data for Forecasting

The goal of this notebook is to create the necessary dataframes to do forecasting. This is my attempt at building something to analyze the statistics as time series. This is, perhaps a dumb approach. Here is the idea. I will create aux dataframes that contain the stats of each team. I will then build a dataframe where the contents is the average performance of the team over the last five games. I will make a couple passes at this. For a first pass, I may not include statistics about what they allowed their opponents to do. I am also not be concerned with whether or not they were the home team.

In [1]:
import os

home_dir = "/workspace/nba-study"
csv_dir = "datasets/clean/csv"
os.chdir(home_dir)
current_dir = os.getcwd()
print(current_dir)

/workspace/nba-study


Let's import the relevant packages.

In [2]:
import pandas as pd
import numpy as np

Now we load the relevant data.

In [3]:
from src.utils import get_df

game_data = get_df("game_data_clean",csv_dir)

We did something slightly weird with the dates earlier. Let's fix that. We will add the dates back in first as a string and then as a datetime object in order to use the built in ordering. Eventually, we will probably go earlier into the cleaning process and add these steps.

In [52]:
def add_date(df:pd.DataFrame) -> pd.DataFrame:
    cols = set(df.columns)
    requisite_data = set(['day', 'month', 'year'])
    if not requisite_data.issubset(cols):
        raise ValueError("The requisite data is not present")
    for col in requisite_data:
        df[col] = df.apply(lambda x: str(x[col]), axis=1)
    df['datestr'] = df.apply(lambda x:x['day']+'-'+x['month']+'-'+x['year'], axis=1)
    df['date'] = pd.to_datetime(df['datestr'], format='%d-%m-%Y')
    df.drop(labels=requisite_data, inplace=True,axis=1)
    df.drop(labels=['datestr'],inplace=True,axis=1)
    return df

In [53]:
game_data = add_date(game_data)
game_data.head()

,season_id,team_id_home,team_name_home,wl_home,fgm_home,fga_home,fg3m_home,fg3a_home,ftm_home,fta_home,...,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,date
game_id,,,,,,,,,,,,,,,,,,,,,
28500005,21985,1610612737,Atlanta Hawks,L,41.0,92.0,0.0,3.0,9.0,18.0,...,11.0,29.0,40.0,21.0,11.0,7.0,17.0,19.0,100.0,1985-10-25
28500006,21985,1610612758,Sacramento Kings,L,39.0,88.0,0.0,2.0,26.0,41.0,...,11.0,31.0,42.0,19.0,7.0,7.0,18.0,32.0,108.0,1985-10-25
28500010,21985,1610612765,Detroit Pistons,W,39.0,88.0,0.0,1.0,40.0,49.0,...,13.0,28.0,41.0,27.0,10.0,7.0,20.0,32.0,116.0,1985-10-25
28500011,21985,1610612762,Utah Jazz,L,42.0,82.0,0.0,2.0,24.0,38.0,...,17.0,30.0,47.0,23.0,10.0,7.0,19.0,28.0,112.0,1985-10-25
28500008,21985,1610612744,Golden State Warriors,L,36.0,91.0,0.0,4.0,33.0,52.0,...,13.0,31.0,44.0,26.0,11.0,3.0,22.0,40.0,119.0,1985-10-25


We are also going to reorder the columns so we can see which teams played at a glance. As it is right now, the away team is hidden in the abbreviated part of the dataframe.

In [58]:
print(game_data.columns)
new_order = ['date','team_name_home','pts_home', 'team_name_away','pts_away',
             'team_id_home','team_id_away', 'wl_home', 'fgm_home',
             'fga_home', 'fg3m_home', 'fg3a_home', 'ftm_home', 'fta_home',
       'oreb_home', 'dreb_home', 'reb_home', 'ast_home', 'stl_home',
       'blk_home', 'tov_home', 'pf_home',  'plus_minus_home',
        'fgm_away', 'fga_away', 'fg3m_away',
       'fg3a_away', 'ftm_away', 'fta_away', 'oreb_away', 'dreb_away',
       'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away',
       'season_id']

Index(['season_id', 'team_id_home', 'team_name_home', 'wl_home', 'fgm_home',
       'fga_home', 'fg3m_home', 'fg3a_home', 'ftm_home', 'fta_home',
       'oreb_home', 'dreb_home', 'reb_home', 'ast_home', 'stl_home',
       'blk_home', 'tov_home', 'pf_home', 'pts_home', 'plus_minus_home',
       'team_id_away', 'team_name_away', 'fgm_away', 'fga_away', 'fg3m_away',
       'fg3a_away', 'ftm_away', 'fta_away', 'oreb_away', 'dreb_away',
       'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away',
       'pts_away', 'date'],
      dtype='object')


We are going to collect all of the team ids and make a dataframe for each of them. We will, in general, need to sort these dataframes by date. This may involve making some custom method/function or it may involve creating a column of datetime objects.

We checked that the set of ids is the same whether we use home or away teams.

In [55]:
team_ids = game_data['team_id_home'].unique()

def get_home_games(data,team_id)-> pd.DataFrame:
    return data.query(f"team_id_home == {team_id}")


def get_away_games(data, team_id)-> pd.DataFrame:
    return data.query(f"team_id_away == {team_id}")


def get_all_games(data, team_id) -> pd.DataFrame:
    all_games = get_home_games(data,team_id).append(get_away_games(data,team_id))
    return all_games.sort_values("date",ascending=True)
    


In [56]:
all = get_all_games(game_data,team_1)
home = get_home_games(game_data,team_1)
away = get_away_games(game_data, team_1)
print(home.shape,away.shape,all.shape)


(1574, 38) (1577, 38) (3151, 38)


/tmp/ipykernel_3686/4000299224.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games = get_home_games(data,team_id).append(get_away_games(data,team_id))


In [57]:
all.head()

,season_id,team_id_home,team_name_home,wl_home,fgm_home,fga_home,fg3m_home,fg3a_home,ftm_home,fta_home,...,oreb_away,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,date
game_id,,,,,,,,,,,,,,,,,,,,,
28500005,21985,1610612737,Atlanta Hawks,L,41.0,92.0,0.0,3.0,9.0,18.0,...,11.0,29.0,40.0,21.0,11.0,7.0,17.0,19.0,100.0,1985-10-25
28500013,21985,1610612749,Milwaukee Bucks,W,44.0,85.0,2.0,5.0,27.0,30.0,...,12.0,24.0,36.0,14.0,9.0,7.0,27.0,34.0,91.0,1985-10-26
28500030,21985,1610612737,Atlanta Hawks,W,37.0,86.0,1.0,3.0,27.0,38.0,...,19.0,25.0,44.0,15.0,8.0,3.0,12.0,26.0,87.0,1985-10-29
28500045,21985,1610612738,Boston Celtics,W,37.0,78.0,1.0,5.0,34.0,39.0,...,14.0,25.0,39.0,22.0,12.0,2.0,14.0,35.0,105.0,1985-11-01
28500054,21985,1610612737,Atlanta Hawks,W,44.0,86.0,0.0,0.0,26.0,37.0,...,19.0,29.0,48.0,22.0,7.0,2.0,16.0,33.0,113.0,1985-11-02
